## 카카오 API로 가게 정보 불러오기

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import itertools
import requests

In [2]:
jen_region = ['대학로', '북촌', '서촌', '홍대입구역', '합정', '연남', '해방촌', '경리단길', '종로3가', '성수동', '가로수길', '송리단길', '명동', '신림', '문래', '이수역', '용리단길']
non_jen_region = ['강남역', '압구정로데오', '건대입구역', '여의도역', '동대문', '잠실역', '회기역', '영등포', '청담', '사당']
common_region = ['성내동', '시흥동', '삼양동', '중화동',  '월계동', '응암동', '신도림동', '신정동', '도봉동', '방화동']
regions = jen_region + non_jen_region + common_region
category = ['FD6', 'CE7', 'AT4', 'CT1'] # FD6 = 음식점, CE7 = 카페, AT4 = 관광명소, CT1 = 문화시설
pages = [i for i in range(1, 10)]

In [ ]:
# 지역 개수 확인
print(len(jen_region))
print(len(non_jen_region))
print(len(common_region))
print(len(regions))
print(regions)

17
10
10
37
['대학로', '북촌', '서촌', '홍대입구역', '합정', '연남', '해방촌', '경리단길', '종로3가', '성수동', '가로수길', '송리단길', '명동', '신림', '문래', '이수역', '용리단길', '강남역', '압구정로데오', '건대입구역', '여의도역', '동대문', '잠실역', '회기역', '영등포', '청담', '사당', '성내동', '시흥동', '삼양동', '중화동', '월계동', '응암동', '신도림동', '신정동', '도봉동', '방화동']


In [ ]:
REST_API_KEY = "REST_API_KEY"

In [5]:
headers = {
    "Authorization": f"KakaoAK {REST_API_KEY}"
}

In [6]:
store_url = "https://dapi.kakao.com/v2/local/search/keyword?query={}&category_group_code={}&page={}"

In [7]:
def request_all_review(request_url):
    try:
        # 요청 보내기
        response = requests.get(request_url, headers=headers).json()
        
        # 응답 구조 확인
        if 'comment' not in response:
            print(f"Error: 'comment' key not found in response. Response: {response}")
            return [], 0, False
        
        comment_data = response['comment']
        
        # 댓글 목록이 비어 있는지 확인
        comment_list = comment_data.get('list', [])
        if not comment_list:
            print("Warning: No comments in this response.")
            return [], 0, False
        
        # 다음 페이지 여부와 마지막 댓글 ID 추출
        has_next_flag = comment_data['hasNext']
        last_comment_id = comment_list[-1]['commentid']
        
        return comment_list, last_comment_id, has_next_flag
    
    except Exception as e:
        print(f"Error occurred while fetching reviews: {e}")
        return [], 0, False

In [8]:
comment_url = "https://place.map.kakao.com/m/commentlist/v/{}/{}?order=USEFUL&onlyPhotoComment=false"

In [ ]:
all_comments = []

for region in regions:
    region_comments = []
    for cat in category:
        for page in pages:
            print(region, cat, page)
            request_url = store_url.format(region, cat, page)
            response = requests.get(request_url, headers=headers).json()
            datas = response['documents']
            is_end = response['meta']['is_end']
            ids = [data['id'] for data in datas]
            # print(ids)
            # print(is_end)
            
            for id in ids:
                last_comment_id = 0
                has_next_flag = True

                while has_next_flag:
                    request_review_again_url = comment_url.format(id, last_comment_id)
                    print(request_review_again_url)
                    comment_list, last_id, has_next = request_all_review(request_review_again_url)
                    region_comments.extend(comment_list)
                    last_comment_id = last_id
                    has_next_flag = has_next
                    # print(comment_list)
                    # print("########")
            
            if is_end: # 현재 페이지가 마지막 페이지인지 여부
                break
    
    all_comments.extend(region_comments)

In [ ]:
# Slack Webhook URL
slack_webhook_url = "SLACK_WEBHOOK_URL"  # Replace with your Slack Webhook URL

all_comments = []

# Define the fetch_comments function
def fetch_comments(region, cat, page):
    region_comments = []  # Initialize region-specific comment list
    print(region, cat, page)
    
    request_url = store_url.format(region, cat, page)
    response = requests.get(request_url, headers=headers).json()
    datas = response['documents']
    is_end = response['meta']['is_end']
    ids = [data['id'] for data in datas]

    for id in ids:
        last_comment_id = 0
        has_next_flag = True

        while has_next_flag:
            request_review_again_url = comment_url.format(id, last_comment_id)
            print(request_review_again_url)
            comment_list, last_id, has_next = request_all_review(request_review_again_url)
            region_comments.extend(comment_list)  # Add comments to region-specific list
            last_comment_id = last_id
            has_next_flag = has_next

    # Once all comments for this region are collected, append them to all_comments
    all_comments.extend(region_comments)

    if is_end:  # Check if the current page is the last page
        return  # End processing when the current page is the last page

def send_slack_notification(message):
    payload = {
        "text": message
    }
    response = requests.post(slack_webhook_url, json=payload)
    if response.status_code != 200:
        print(f"Failed to send Slack notification: {response.status_code}, {response.text}")
    else:
        print("Slack notification sent successfully.")


for region in jen_region:
    for cat in category:
        for page in pages:
            fetch_comments(region, cat, page)

# Send a Slack notification once the crawling is complete
send_slack_notification("Crawling process has been completed successfully!")



대학로 FD6 1
https://place.map.kakao.com/m/commentlist/v/922672769/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/27092640/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/24073304/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/24828254/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/182830811/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/1076663063/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/1145270203/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/907451099/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/16288291/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/10618939/0?order=USEFUL&onlyPhotoComment=false
https://place.map.kakao.com/m/commentlist/v/1367022330/0?order=USEFUL&onl

In [ ]:
print(all_comments)

In [ ]:
print(len(all_comments) )